In [2]:
# default_exp cli

# papersweep

> Combining papermill and wandb sweeps for fricitionless experimebnts with Jupyter notebooks

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#export
from fastcore.all import *
import yaml
import wandb
import papermill as pm

In [5]:
#export
@call_parse
def papersweep_exec(input_nb:Param("Input notebook", str),
                    sweep_config:Param("YAML file with the sweep config", str), 
                    entity:Param("wandb entity", str), 
                    project:Param("wandb project", str),
                    pm_params:Param("YAML file with papermill parameters", str)=None,
                    sweep_id:Param("Sweep ID. This option omits `sweep_config`", str)=None, 
                    login_key:Param("Login key for wandb", str)=None):
    if login_key:
        wandb.login(key=login_key)
    with maybe_open(sweep_config, 'r') as f:
        sc = yaml.safe_load(f)
    if pm_params:
        with maybe_open(pm_params, 'r') as f:
            _pm_params = yaml.safe_load(f)
    sid = wandb.sweep(sweep=sc, entity=entity, project=project) if not sweep_id else sweep_id
    sweep_agent = wandb.agent(sid,
                              function=partial(pm.execute_notebook,  
                                               input_path=input_nb, 
                                               output_path='__.ipynb', 
                                               parameters=_pm_params))
    return sid

In [6]:
a = papersweep_exec(input_nb='./_example_tsai.ipynb', 
               sweep_config='examples/sweep_config.yaml',
               entity='vrodriguezf', 
               project='papersweep', 
               pm_params='examples/run_config.yaml')

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


KeyboardInterrupt: Interrupted by user

In [17]:
with maybe_open('examples/run_config.yaml', 'r') as f:
    rc = yaml.safe_load(f)

In [36]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_cli.ipynb.
Converted index.ipynb.
